Running in pre-compiled env `/cvmfs/icecube.opensciencegrid.org/py3-v4.1.0/RHEL_7_x86_64/metaprojects/combo/stable/env-shell.sh` \
Get grb list from GRBWeb \
Start date and end date are aligned with GRECO v2.4 data

In [1]:
import numpy as np
import sqlite3
import pandas
import os

In [2]:
# Download the SQLite file from the GRBweb webpage
# os.system("wget -O /home/cjchen/2020-2021/Spring2021/greco_grb/data/grbweb/GRBweb2.sqlite https://icecube.wisc.edu/~grbweb_public/GRBweb2.sqlite")


In [3]:
db = sqlite3.connect('/home/cjchen/2020-2021/Spring2021/greco_grb/data/grbweb/GRBweb2.sqlite')
db

In [4]:
# Print the names of all the tables
table_names = pandas.read_sql_query("SELECT * from sqlite_sequence", db)
table_names


,name,seq
0,BATSE,2702
1,BeppoSAX,1082
2,Fermi_GBM,2965
3,Fermi_GBM_precursor_Zhu,239
4,Fermi_GBM_subthreshold,4205
5,Fermi_LAT,146
6,GCN_circulars,17236
7,IPN,2163
8,Summary,7432
9,Swift,1430


In [5]:
# Get numpy arrays containing the right ascension, declination, mjd, ... of the entries in the Summary table
Summary_table = pandas.read_sql_query("SELECT * from Summary", db)
Summary_table.head()

,id,GRB_name,GRB_name_Fermi,T0,T0_source,ra,ra_source,decl,decl_source,pos_error,...,fluence,fluence_source,fluence_error,fluence_error_source,redshift,redshift_source,T100,GBM_located,mjd,mjd_source
0,1,GRB150702A,GRB150702998,86197.55,Fermi_LAT,52.780000,Fermi_LAT,-57.000000,Fermi_LAT,0.360000,...,1.174100e-05,Fermi_GBM,5.402800e-08,Fermi_GBM,NaN,None,48.17,0,57205.997657,Fermi_LAT
1,2,GRB080120A,None,62910.00,"GCN_circulars,INTEGRAL",225.287500,IPN,-10.900000,IPN,0.051667,...,NaN,None,NaN,None,NaN,None,NaN,0,54485.728125,"GCN_circulars,INTEGRAL"
2,3,GRB080123A,None,15717.00,"GCN_circulars,Swift Det",338.943083,Swift,-64.900889,Swift,0.000220,...,5.700000e-07,Swift,NaN,None,NaN,None,115.00,0,54488.181910,"GCN_circulars,Swift Det"
3,4,GRB080121A,None,77395.00,Swift,137.235000,Swift,41.841000,Swift,0.023300,...,3.000000e-08,Swift,NaN,None,0.046,"GCN_circulars,Swift-BAT Det",0.70,0,54486.895775,Swift
4,5,GRB080122A,None,66764.00,"GCN_circulars,IPN Triangulation",191.350000,IPN,27.016667,IPN,1.666667,...,NaN,None,NaN,None,NaN,None,NaN,0,54487.772731,"GCN_circulars,IPN Triangulation"


In [6]:
keys = Summary_table.keys()
print("Keys in the 'Summary' table: {}\n".format(list(keys)))
RA  = np.array(Summary_table.ra)   # right ascension
DEC = np.array(Summary_table.decl) # declination
MJD = np.array(Summary_table.mjd)  # modified julian date
print(" RA: {}".format(RA[:3]))
print(" DEC: {}".format(DEC[:3]))
print(" MJD: {}".format(MJD[:3]))

Summary_table.shape

Keys in the 'Summary' table: ['id', 'GRB_name', 'GRB_name_Fermi', 'T0', 'T0_source', 'ra', 'ra_source', 'decl', 'decl_source', 'pos_error', 'pos_error_source', 'T90', 'T90_source', 'T90_error', 'T90_error_source', 'T90_start', 'T90_start_source', 'fluence', 'fluence_source', 'fluence_error', 'fluence_error_source', 'redshift', 'redshift_source', 'T100', 'GBM_located', 'mjd', 'mjd_source']

 RA: [ 52.78       225.2875     338.94308333]
 DEC: [-57.         -10.9        -64.90088889]
 MJD: [57205.99765683 54485.728125   54488.18190972]


(7432, 27)

## GRECO v2.4
`/data/user/mlarson/combo_r129072/scripts/greco_online/skylab_dataset/greco_online/current` \
Data 2012 - 2018 \
from MJD `56043.42313825331` to `58648.87564821623` \
Or `2012-04-26 10:09:19.145 UTC` ~ `2019-06-14 21:00:56.006 UTC` \
No boundary constraint here.

In [7]:
if np.isnan(np.sum(MJD)):
    print("We have null MJD")
else:
    print("All bursts have MJD")

All bursts have MJD


All angles in degrees \
All times in days

In [19]:
query = """
SELECT GRB_name AS grb_name
, GRB_name_Fermi AS grb_name_GBM
, ra
, decl AS dec
, mjd AS mjd_start
, T100/86400. AS t100
, T90/86400. AS t90
, GBM_located AS gbm
, pos_error AS source_uncertainty
, redshift
, (mjd + 0.5*T100/86400.) AS t_center
FROM Summary
WHERE (T100 IS NOT NULL) And (t_center between 56043.42313825331 And 58648.87564821623)
ORDER BY grb_name DESC
"""
grb_table = pandas.read_sql_query(query, db)
grb_table.head()

,grb_name,grb_name_GBM,ra,dec,mjd_start,t100,t90,gbm,source_uncertainty,redshift,t_center
0,GRB190613B,GRB190613449,305.431917,-4.646667,58647.449285,0.001905,0.000056,0,0.000056,NaN,58647.450238
1,GRB190613A,GRB190613172,182.529208,67.235278,58647.171736,0.000204,0.000198,0,0.000056,NaN,58647.171838
2,GRB190612A,GRB190612165,223.950000,62.100000,58646.164836,0.001677,0.001677,1,7.025096,NaN,58646.165675
3,GRB190611B,GRB190611950,84.758333,50.833333,58645.949833,0.001217,0.001164,0,5.900000,NaN,58645.950442
4,GRB190611A,None,324.722625,-56.074278,58645.742373,0.000484,0.000484,0,0.000194,NaN,58645.742615


In [20]:
grb_table.shape

(2051, 11)

In [21]:
type(grb_table)

pandas.core.frame.DataFrame

In [22]:
# grb_table.to_pickle("/home/cjchen/2020-2021/Spring2021/greco_grb/data/grbweb/grb_grbweb.pkl")